In [ ]:
import numpy as np

data=np.array([1,2,3,2,2,1,2,3,2,45])

mean=data.mean()
std=data.std()

z_score=(data-mean)/std

print('Data ',data)
print('Z score ',np.round(z_score,3))

outliers=data[np.abs(z_score)>=2.9]     #absoloute values
print('\n Outliers (|Z|>= 3)',outliers)

Data  [ 1  2  3  2  2  1  2  3  2 45]
Z score  [-0.41  -0.333 -0.256 -0.333 -0.333 -0.41  -0.333 -0.256 -0.333  2.996]

 Outliers (|Z|>= 3) [45]


In [17]:
data=np.array([-7,1,2,3,4,2,1,2,3,2,10])

Q1=np.percentile(data,25)
Q3=np.percentile(data,75)
IQR=Q3-Q1   #interquartile range

lower=Q1-1.5*IQR
upper=Q3+1.5*IQR

outliers=data[(data<lower) | (data>upper)]
print(outliers)

print(f'first quartile {Q1}, third quartile {Q3}, IQR {IQR}')
print(f'Lower bound={lower}, Upper bound={upper}')
print('Outliers by IQR(Interquartile Range) method',outliers)

[-7 10]
first quartile 1.5, third quartile 3.0, IQR 1.5
Lower bound=-0.75, Upper bound=5.25
Outliers by IQR(Interquartile Range) method [-7 10]


In [19]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [20]:
df=pd.read_csv('creditcard.csv')


In [21]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [22]:
df.columns


Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [27]:
df['Class'].value_counts(normalize=True)

Class
0    0.998273
1    0.001727
Name: proportion, dtype: float64

In [28]:
df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,-1.040458,-0.031513,-0.188093,-0.084316,0.041333,-0.302620,-0.660377,0.167430,-0.256117,0.382948,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [29]:
df.drop(columns='Time',axis=1,inplace=True)

In [30]:
df.head(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0


In [32]:
df['Amount']=StandardScaler().fit_transform(df[['Amount']])

In [33]:
df.sample(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
139231,-2.411781,0.899072,1.205213,0.077283,-1.418744,0.277203,-1.266536,1.629570,-0.468653,-0.748712,1.000830,1.352855,0.789902,0.684418,0.832294,0.983632,-0.183523,0.694446,-0.528899,-0.213434,0.499611,1.096599,-0.099294,0.088109,-0.308983,0.430817,-0.182400,-0.164312,-0.205300,0
58004,1.265650,-0.033962,0.339746,0.816681,0.112505,1.009188,-0.532845,0.260891,0.588923,-0.006044,-0.612224,0.740374,0.571646,-0.258727,-0.560643,0.584572,-1.086778,1.010519,0.706729,-0.066137,0.027038,0.327129,-0.362913,-1.346774,0.867495,-0.126562,0.057891,0.001751,-0.349231,0


In [ ]:
#Isolation forest

from sklearn.ensemble import IsolationForest
#call
iso_forest=IsolationForest(contamination=0.0017,random_state=42)

In [39]:
X=df.drop(columns='Class',axis=1)
y=df['Class']

In [40]:
df['anomaly']=iso_forest.fit_predict(X)


In [44]:
df.sample(5)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,anomaly
96740,1.127670,0.218605,0.276857,0.476889,0.071266,0.196710,-0.164790,0.214539,-0.254962,-0.206102,2.106592,1.167247,0.231316,-0.059779,0.885473,0.078017,0.299289,-0.725912,-0.633470,-0.127025,-0.178808,-0.474732,0.176330,-0.323548,0.043194,0.135920,0.007301,0.013996,-0.349271,0,1
250036,2.076595,-0.234088,-1.929690,-0.086358,0.687400,0.026896,-0.007536,-0.001220,0.588676,0.201186,-0.109997,-0.066778,-1.116443,0.874665,0.351701,0.313318,-1.023156,0.877502,0.391964,-0.230623,0.222616,0.650484,-0.094972,-0.356280,0.330759,-0.090214,-0.037786,-0.073101,-0.289300,0,1
139228,-0.814337,0.316373,0.962254,-3.222609,2.140760,3.531675,0.098741,0.675510,1.604610,-0.884595,-0.203527,0.102285,-0.228267,-0.691781,0.940373,-0.158075,-1.089043,0.635488,0.358493,0.419405,-0.196873,-0.036675,-0.465892,0.999526,0.731593,-0.794049,0.134655,-0.254810,-0.349231,0,1
40978,1.090887,0.054321,0.390538,1.234866,0.002596,0.580117,-0.195062,0.332997,0.190653,0.034614,1.210693,0.736331,-1.313471,0.503978,-0.423805,-0.776371,0.359387,-0.810482,-0.434823,-0.296054,-0.038317,0.090412,-0.024726,-0.308675,0.524262,-0.295697,0.047226,-0.001231,-0.349231,0,1
136862,0.261376,-0.308606,1.476005,-1.447583,-0.677457,-0.314552,-0.220692,-0.345320,-2.514749,1.482197,-0.062078,-0.481454,1.840433,-0.771418,0.927718,-1.330392,0.938589,-0.382754,0.470447,-0.054820,-0.071257,0.511067,0.027317,0.108263,-0.894414,-0.213054,0.094431,-0.028219,-0.333239,0,1


In [ ]:
df['anomaly'].value_counts()
#Class== 0=no fraud , 1=fraud
#anomaly== 1=no fraud, -1=fraud

anomaly
 1    284322
-1       485
Name: count, dtype: int64

In [46]:
df['anomaly']=df['anomaly'].map({1:0,-1:1})     #dictinary

In [47]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,anomaly
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0,0


In [48]:
from sklearn.metrics import classification_report

print(classification_report(df['Class'],df['anomaly']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    284315
           1       0.29      0.28      0.28       492

    accuracy                           1.00    284807
   macro avg       0.64      0.64      0.64    284807
weighted avg       1.00      1.00      1.00    284807



In [49]:
#One class SVM
from sklearn.svm import OneClassSVM
oc_svm=OneClassSVM(kernel='rbf',gamma='auto',nu=0.002)


In [51]:
X1=df.drop(['Class','anomaly'],axis=1)

In [52]:
df['anomaly_svm']=oc_svm.fit_predict(X1)

In [53]:
df['anomaly_svm']=df['anomaly_svm'].map({1:0,-1:1}) 

In [56]:
print(classification_report(df['Class'],df['anomaly_svm']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    284315
           1       0.10      0.28      0.15       492

    accuracy                           0.99    284807
   macro avg       0.55      0.64      0.57    284807
weighted avg       1.00      0.99      1.00    284807



In [ ]:
#data loading 
#data preprocessing
#Change categorical columns to numeric columns using label encoder
#Create new columns where normal is 0 and anomaly is 1
#Split the data into X and y
#Train/Test split
